# Libaries

In [ ]:
import pandas as pd

# Read and split data

In [ ]:
df = pd.read_parquet("../data/feature/stock_eod_features.parquet")

# Split dataset based on time - keep 10% test for final model evaluation
cutoff_date = df["date"].quantile(0.9)
print("Train/Test cutoff date:", cutoff_date.strftime('%Y-%m-%d'))

train_df = df[df["date"] < cutoff_date].copy()
test_df  = df[df["date"] >= cutoff_date].copy()

In [ ]:
# Exclude non-feature columns
exclude = ['date', 'symbol', 'target']

feat_cols = [c for c in train_df.columns 
             if c not in exclude]

corr_with_target = train_df[feat_cols + ['target']].corr()['target'].sort_values(ascending=False)
corr_with_target

# Preprocessing

In [ ]:
train_df["symbol"] = train_df["symbol"].astype("category")
test_df["symbol"]  = test_df["symbol"].astype("category")

train_df = pd.get_dummies(train_df, columns=["symbol"], drop_first=False)
test_df  = pd.get_dummies(test_df,  columns=["symbol"], drop_first=False)

train_df = train_df.reindex(sorted(train_df.columns), axis=1)
test_df  = test_df.reindex(sorted(test_df.columns), axis=1)

train_df.head()

In [ ]:
target_column = 'target'
exclude = ['date', target_column]

feature_columns = [column for column in train_df.columns if column not in exclude]

X_train = train_df[feature_columns].values
y_train = train_df[target_column].values

X_test = test_df[feature_columns].values
y_test = test_df[target_column].values

# Baseline model